In [1]:
import pandas as pd
import os
import numpy as np
import random
import json
import pickle as pkl
import matplotlib.pyplot as plt
import scipy.sparse as sp
from math import log
from sklearn import svm
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
from collections import defaultdict
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.optimizer import Adam
import pgl

In [2]:
#Location of each resource type
#['category'][0]['coding'][0]['display'], Careplan

#['type'][0]['text'], encounter
#['vaccineCode']['coding'][0]['display'], immunization
#['medicationCodeableConcept']['coding'][0]['display'], medication
#['type']['coding'][0]['display']
#['id'], patient
#['code']['coding'][0]['display'], Procedure
#['code']['coding'][0]['display'], condition
#['code']['coding'][0]['display'], observation
#['code']['coding'][0]['display'], diagonstic report

#['vaccineCode']['coding'][0]['display'], immunization
#['medicationCodeableConcept']['coding'][0]['display'], medication


# Corpus preparation

### Change the SYNTHEA STU-3 FHIR data to pandas dataframe 

In [3]:
directory = os.fsencode('../SYNTHEA/')
encounters,conditions,patient_ids,procedures, observations, dignotstics_reports, immunizations, medication_orders, careplans = ([] for _ in range(9))

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    with open('../SYNTHEA/'+filename, "r") as read_file:
        data = json.load(read_file)
        resources = data['entry']
        patient_id = resources[0]['resource']['id']
        patient_ids.append(patient_id)
        patient_encounters,patient_conditions, patient_procedures,patient_observations, patient_diagnostics_reports, patient_immunizations, patient_medication_orders, patient_careplans = (set() for _ in range(8))
   
        for resource in resources:
            resource_type = resource['resource']['resourceType']
            if resource_type =='Encounter':
                patient_encounters.add(resource['resource']['type'][0]['coding'][0]['code'])
            elif resource_type =='Condition':
                patient_conditions.add(resource['resource']['code']['coding'][0]['code'])
            elif resource_type =='Procedure':
                code = resource['resource']['code']['coding'][0]['code']
                #ignore documentation of current medications
                if code!='428191000124101':
                    patient_procedures.add(code)  
            elif resource_type =='Observation':
                patient_observations.add(resource['resource']['code']['coding'][0]['code'])
            elif resource_type =='DiagnosticReport':
                patient_diagnostics_reports.add(resource['resource']['code']['coding'][0]['code'])
            elif resource_type == 'Immunization':
                patient_immunizations.add(resource['resource']['vaccineCode']['coding'][0]['code'])
            elif resource_type == 'MedicationOrder':
                patient_medication_orders.add(resource['resource']['medicationCodeableConcept']['coding'][0]['code'])
            elif resource_type == 'CarePlan':
                patient_careplans.add(resource['resource']['category'][0]['coding'][0]['code'])
        encounters.append(','.join(patient_encounters))
        conditions.append(','.join(patient_conditions))
        procedures.append(','.join(patient_procedures))
        observations.append(','.join(patient_observations))
        dignotstics_reports.append(','.join(patient_diagnostics_reports))
        immunizations.append(','.join(patient_immunizations))
        medication_orders.append(','.join(patient_medication_orders))
        careplans.append(','.join(patient_careplans))
        
        
data = {'patient': patient_ids,'Encounter': encounters, 'Condition': conditions, 'Procedure':procedures, 'Observation':observations, 
        'Diagnostics Report':dignotstics_reports, 'Immunization':immunizations, 'Medication Orders':medication_orders, 'Careplan': careplans}
resource_df = pd.DataFrame(data)

In [4]:
resource_df.head()

patient  \
0  9f59ec59-5116-401b-bcbb-f019a6ca7a4f   
1  765382db-8551-4733-8658-d9fc7b0f7ee8   
2  63927398-3432-4526-9960-0da4359e3be2   
3  7af203fe-86af-42a9-b6ee-f2c2828ead13   
4  9a0ea485-3bba-457f-bcaa-8cbb8fba34ef   

                                          Encounter  \
0  185349003,170258001,185345009,50849002,183495009   
1                               185349003,185345009   
2                     185349003,170258001,185345009   
3                               185349003,185345009   
4            185349003,50849002,183452005,185345009   

                                           Condition  \
0  444470001,444814009,195662009,44465007,3598170...   
1  368581000119106,444814009,38341003,15777000,10...   
2                       275272006,43878008,444814009   
3                        40055000,38341003,444814009   
4  74400008,444814009,38341003,15777000,62106007,...   

                       Procedure  \
0  305428000,699253003,269911007   
1                       23426006   
2                      117015009   
3                                  
4                       80146002   

                                         Observation Diagnostics Report  \
0              8302-2,39156-5,55284-4,8331-1,29463-7                      
1  2947-0,8331-1,38483-4,49765-1,29463-7,18262-6,...    57698-3,51990-0   
2              8302-2,39156-5,55284-4,8331-1,29463-7                      
3  33914-3,8302-2,39156-5,55284-4,2085-9,2093-3,2...            57698-3   
4  2947-0,8331-1,38483-4,49765-1,29463-7,18262-6,...    57698-3,51990-0   

     Immunization            Medication Orders                     Careplan  
0  115,140,114,62  198405,310965,857005,608680  385691007,53950000,91251008  
1         140,113                860975,608680           698360004,53950000  
2  140,113,114,62                834101,824184                               
3         140,113                       824184                               
4     121,140,113                       860975           698360004,47387005

In [5]:
resource_df.columns

Index(['patient', 'Encounter', 'Condition', 'Procedure', 'Observation',
       'Diagnostics Report', 'Immunization', 'Medication Orders', 'Careplan'],
      dtype='object')

In [6]:
res = resource_df.replace('', np.nan)

In [7]:
res.isnull().sum()

patient                 0
Encounter               1
Condition              47
Procedure             393
Observation             3
Diagnostics Report    312
Immunization          209
Medication Orders      91
Careplan              156
dtype: int64

In [8]:
#drop patients with null values for datasets understudy
res_condition = res.dropna(axis=0, how='any', subset=['Condition','Procedure', 'Medication Orders', 'Observation'])

In [9]:
res_condition.shape

(593, 9)

In [10]:
res_condition.isnull().sum()

patient                 0
Encounter               0
Condition               0
Procedure               0
Observation             0
Diagnostics Report    211
Immunization            2
Medication Orders       0
Careplan               24
dtype: int64

In [11]:
diseases = []
disease_bundled = []
for condition in res_condition['Condition'].values:
    diseases.extend(condition.split(','))
    disease_bundled.append(condition.split(','))

In [12]:
len(diseases)

2904

In [13]:
from collections import Counter

disease_count = Counter(diseases)

In [14]:
disease_count

Counter({'444470001': 5,
         '444814009': 364,
         '195662009': 250,
         '44465007': 54,
         '359817006': 7,
         '10509002': 263,
         '368581000119106': 11,
         '38341003': 164,
         '15777000': 176,
         '44054006': 24,
         '275272006': 3,
         '43878008': 106,
         '74400008': 31,
         '62106007': 34,
         '422034002': 6,
         '428251008': 31,
         '33737001': 12,
         '39848009': 34,
         '72892002': 88,
         '429007001': 18,
         '398254007': 9,
         '410429000': 18,
         '230690007': 28,
         '239873007': 29,
         '195967001': 18,
         '367498001': 10,
         '65363002': 72,
         '233678006': 21,
         '26929004': 13,
         '233604007': 3,
         '423121009': 7,
         '162573006': 11,
         '254637007': 10,
         '82423001': 4,
         '15724005': 1,
         '40055000': 104,
         '403190006': 18,
         '284549007': 18,
         '35999006': 7,


In [15]:
disease_count_sorted = dict(sorted(disease_count.items(), key= lambda item: item[1]))
cases = list(disease_count_sorted.keys())
count = list(disease_count_sorted.values())
fig = plt.figure(figsize=(30,5))
plt.bar(cases, count)

INFO 2022-08-19 10:35:30,895 category.py:227] Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO 2022-08-19 10:35:30,897 category.py:227] Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.


<BarContainer object of 109 artists>

In [16]:
disease_count_sorted

{'15724005': 1,
 '287182007': 1,
 '86849004': 1,
 '262574004': 1,
 '157141000119108': 1,
 '97331000119101': 1,
 '230265002': 1,
 '69896004': 1,
 '67841000119103': 1,
 '254632001': 1,
 '609496007': 1,
 '196416002': 1,
 '1501000119109': 2,
 '239720000': 2,
 '93761005': 2,
 '110030002': 2,
 '363406005': 2,
 '37849005': 2,
 '275272006': 3,
 '233604007': 3,
 '90781000119102': 3,
 '425048006': 3,
 '239872002': 3,
 '124171000119105': 3,
 '82423001': 4,
 '403192003': 4,
 '444448004': 4,
 '90560007': 4,
 '444470001': 5,
 '109838007': 5,
 '197927001': 5,
 '1551000119108': 5,
 '55680006': 5,
 '422034002': 6,
 '198992004': 6,
 '359817006': 7,
 '423121009': 7,
 '35999006': 7,
 '307731004': 7,
 '30832001': 7,
 '127013003': 7,
 '40275004': 7,
 '85116003': 8,
 '62564004': 8,
 '47693006': 8,
 '84757009': 8,
 '398254007': 9,
 '367498001': 10,
 '254637007': 10,
 '399211009': 10,
 '22298006': 10,
 '368581000119106': 11,
 '162573006': 11,
 '33737001': 12,
 '713197008': 12,
 '38822007': 12,
 '201834006': 12

###  Choose conditions for study

In [17]:
candidates = list(disease_count_sorted.keys())[-5:]
print(candidates)

['38341003', '15777000', '195662009', '10509002', '444814009']


In [18]:
from collections import defaultdict
case_intersection = defaultdict(lambda: [])
for case1 in candidates:
    for case2 in candidates:
        count = 0
        for case in disease_bundled:
            if case1 in case and case2 in case:
                count+=1
        case_intersection[case1].append(count)
                
                
    

In [19]:
#Check comorbidity between candidates for study
case_intersection_df = pd.DataFrame(case_intersection, index = list(case_intersection.keys()))

In [20]:
case_intersection_df

38341003  15777000  195662009  10509002  444814009
38341003        164        55         69        68        108
15777000         55       176         72        77        106
195662009        69        72        250       119        158
10509002         68        77        119       263        166
444814009       108       106        158       166        364

In [21]:
case_intersection_df.values[[np.arange(case_intersection_df.shape[0])]*2] = 0

/var/folders/9n/gzwg3s7j7pn060z120b3pb_h0000gn/T/ipykernel_1967/2729502969.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  case_intersection_df.values[[np.arange(case_intersection_df.shape[0])]*2] = 0


In [22]:
case_intersection_df

38341003  15777000  195662009  10509002  444814009
38341003          0        55         69        68        108
15777000         55         0         72        77        106
195662009        69        72          0       119        158
10509002         68        77        119         0        166
444814009       108       106        158       166          0

In [23]:
case_intersection_df.sum(axis=1)

38341003     300
15777000     310
195662009    418
10509002     430
444814009    538
dtype: int64

In [24]:
candidates

['38341003', '15777000', '195662009', '10509002', '444814009']

In [25]:
common = []
for case in disease_bundled:
    common.append(np.in1d(np.array(candidates), np.array(case)).sum())

In [26]:
unique_indicies = []
for i in range(len(common)):
    if common[i]==1:
        unique_indicies.append(i)
        

In [27]:
len(unique_indicies)

143

In [28]:
selected_df = res_condition.iloc[unique_indicies]

In [29]:
selected_df.shape

(143, 9)

In [30]:
selected_df.head()

patient  \
2   63927398-3432-4526-9960-0da4359e3be2   
5   77e05358-bfd9-4399-a474-1fab36f31975   
9   fe1bba02-f021-4e12-81da-2b8f42988574   
21  e223f58a-7029-4287-8e91-83b67336c3da   
24  6979463f-b1ab-48c7-b560-b0b1782aa82e   

                                            Encounter  \
2                       185349003,170258001,185345009   
5   185349003,424441002,170258001,698314001,508490...   
9    185349003,698314001,185345009,50849002,183495009   
21                      185347001,170258001,185345009   
24             185349003,185347001,50849002,308646001   

                                            Condition  \
2                        275272006,43878008,444814009   
5   33737001,39848009,38341003,72892002,429007001,...   
9                                  444470001,10509002   
21                                233678006,444814009   
24             423121009,15777000,162573006,254637007   

                                            Procedure  \
2                                           117015009   
5               274474001,66348005,31208007,252160004   
9                                 699253003,269911007   
21                                395142003,171231001   
24  173160006,703423002,73761001,418891003,3992080...   

                                          Observation  \
2               8302-2,39156-5,55284-4,8331-1,29463-7   
5              33914-3,8302-2,39156-5,55284-4,29463-7   
9   8302-2,39156-5,55284-4,2085-9,2093-3,29463-7,2...   
21  6158-0,29463-7,6248-9,8302-2,7258-7,6206-7,618...   
24  20565-8,2947-0,8302-2,69453-9,6298-4,39156-5,5...   

         Diagnostics Report                      Immunization  \
2                       NaN                    140,113,114,62   
5                       NaN                    115,140,114,62   
9                   57698-3                           140,113   
21                      NaN  10,133,119,83,08,03,140,20,49,21   
24  69409-1,57698-3,51990-0                           140,113   

                             Medication Orders  \
2                                834101,824184   
5                  313782,198405,310965,757594   
9   749762,1020137,861467,849574,834060,831533   
21                 665078,727316,895994,745679   
24                              1736854,583214   

                                             Careplan  
2                                                 NaN  
5   91251008,698358001,385691007,872781000000100,1...  
9                                   53950000,91251008  
21                          326051000000105,699728000  
24                                698360004,182964004

In [31]:
selected_df.isnull().sum()

patient                0
Encounter              0
Condition              0
Procedure              0
Observation            0
Diagnostics Report    72
Immunization           1
Medication Orders      0
Careplan              11
dtype: int64

### format the corpus

In [32]:
condition_df = selected_df[['patient','Procedure', 'Medication Orders', 'Condition']]

In [33]:
condition_df.isnull().sum()

patient              0
Procedure            0
Medication Orders    0
Condition            0
dtype: int64

In [34]:
condition_df.head()

patient  \
2   63927398-3432-4526-9960-0da4359e3be2   
5   77e05358-bfd9-4399-a474-1fab36f31975   
9   fe1bba02-f021-4e12-81da-2b8f42988574   
21  e223f58a-7029-4287-8e91-83b67336c3da   
24  6979463f-b1ab-48c7-b560-b0b1782aa82e   

                                            Procedure  \
2                                           117015009   
5               274474001,66348005,31208007,252160004   
9                                 699253003,269911007   
21                                395142003,171231001   
24  173160006,703423002,73761001,418891003,3992080...   

                             Medication Orders  \
2                                834101,824184   
5                  313782,198405,310965,757594   
9   749762,1020137,861467,849574,834060,831533   
21                 665078,727316,895994,745679   
24                              1736854,583214   

                                            Condition  
2                        275272006,43878008,444814009  
5   33737001,39848009,38341003,72892002,429007001,...  
9                                  444470001,10509002  
21                                233678006,444814009  
24             423121009,15777000,162573006,254637007

In [35]:
condition_df.shape

(143, 4)

In [36]:
len(condition_df['patient'].unique())

143

In [37]:
condition_df['Condition']=condition_df['Condition'].map(lambda x: list(set(x.split(',')) & set(candidates))[0])

/var/folders/9n/gzwg3s7j7pn060z120b3pb_h0000gn/T/ipykernel_1967/868148765.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condition_df['Condition']=condition_df['Condition'].map(lambda x: list(set(x.split(',')) & set(candidates))[0])


In [38]:
condition_df.head()

patient  \
2   63927398-3432-4526-9960-0da4359e3be2   
5   77e05358-bfd9-4399-a474-1fab36f31975   
9   fe1bba02-f021-4e12-81da-2b8f42988574   
21  e223f58a-7029-4287-8e91-83b67336c3da   
24  6979463f-b1ab-48c7-b560-b0b1782aa82e   

                                            Procedure  \
2                                           117015009   
5               274474001,66348005,31208007,252160004   
9                                 699253003,269911007   
21                                395142003,171231001   
24  173160006,703423002,73761001,418891003,3992080...   

                             Medication Orders  Condition  
2                                834101,824184  444814009  
5                  313782,198405,310965,757594   38341003  
9   749762,1020137,861467,849574,834060,831533   10509002  
21                 665078,727316,895994,745679  444814009  
24                              1736854,583214   15777000

In [39]:
condition_df[condition_df['Condition']=='444814009']

patient            Procedure  \
2    63927398-3432-4526-9960-0da4359e3be2            117015009   
21   e223f58a-7029-4287-8e91-83b67336c3da  395142003,171231001   
44   df8f4853-e339-4af0-b423-26b7272bb48d   22523008,288086009   
56   6a511630-d43a-4380-9e60-b928f6287d5d  274474001,117015009   
74   5238d34c-0005-4e3a-ae3b-12d260068af0   46706006,117015009   
..                                    ...                  ...   
915  2ee68828-5d6c-4941-bee9-51cc4fb6a1bc            252160004   
917  471c24d2-f53b-4b45-a387-f51a44a542f8  274474001,288086009   
933  2f21c601-71f6-4ab9-ab66-1e227dc0db5e            228557008   
936  f150af1f-f166-4917-9678-685d6327efed  312681000,305428000   
972  84f1cd4b-348e-4af3-bf2a-52cbbab68313  274474001,288086009   

                       Medication Orders  Condition  
2                          834101,824184  444814009  
21           665078,727316,895994,745679  444814009  
44                         313782,834060  444814009  
56          310965,834101,1367439,834060  444814009  
74          834101,897122,1373463,860975  444814009  
..                                   ...        ...  
915                        757594,849574  444814009  
917                        751905,313782  444814009  
933                        849574,834060  444814009  
936  310965,861467,849574,834060,1049221  444814009  
972                        198405,282464  444814009  

[61 rows x 4 columns]

In [40]:
condition_df['Document'] = condition_df['Procedure'].apply(lambda x : ' '.join(x.split(',')))+ ' ' + condition_df['Medication Orders'].apply(lambda x : ' '.join(x.split(',')))   

/var/folders/9n/gzwg3s7j7pn060z120b3pb_h0000gn/T/ipykernel_1967/663233480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condition_df['Document'] = condition_df['Procedure'].apply(lambda x : ' '.join(x.split(',')))+ ' ' + condition_df['Medication Orders'].apply(lambda x : ' '.join(x.split(',')))


In [41]:
condition_df.head()

patient  \
2   63927398-3432-4526-9960-0da4359e3be2   
5   77e05358-bfd9-4399-a474-1fab36f31975   
9   fe1bba02-f021-4e12-81da-2b8f42988574   
21  e223f58a-7029-4287-8e91-83b67336c3da   
24  6979463f-b1ab-48c7-b560-b0b1782aa82e   

                                            Procedure  \
2                                           117015009   
5               274474001,66348005,31208007,252160004   
9                                 699253003,269911007   
21                                395142003,171231001   
24  173160006,703423002,73761001,418891003,3992080...   

                             Medication Orders  Condition  \
2                                834101,824184  444814009   
5                  313782,198405,310965,757594   38341003   
9   749762,1020137,861467,849574,834060,831533   10509002   
21                 665078,727316,895994,745679  444814009   
24                              1736854,583214   15777000   

                                             Document  
2                             117015009 834101 824184  
5   274474001 66348005 31208007 252160004 313782 1...  
9   699253003 269911007 749762 1020137 861467 8495...  
21    395142003 171231001 665078 727316 895994 745679  
24  173160006 703423002 73761001 418891003 3992080...

In [42]:
# with open('data/corpus/disease.clean.txt', 'w') as f:
#     for doc in condition_df['Document']:
#         f.write(doc+'\n')
    

In [43]:
size = len(condition_df)
train_size = int(0.9*size)
test_size = size - train_size
print(size, train_size, test_size)

143 128 15


In [44]:
condition_df['Condition'].unique() 

array(['444814009', '38341003', '10509002', '15777000', '195662009'],
      dtype=object)

In [45]:
resource_df = condition_df.reset_index(drop=True)

In [46]:
resource_df.head()

patient  \
0  63927398-3432-4526-9960-0da4359e3be2   
1  77e05358-bfd9-4399-a474-1fab36f31975   
2  fe1bba02-f021-4e12-81da-2b8f42988574   
3  e223f58a-7029-4287-8e91-83b67336c3da   
4  6979463f-b1ab-48c7-b560-b0b1782aa82e   

                                           Procedure  \
0                                          117015009   
1              274474001,66348005,31208007,252160004   
2                                699253003,269911007   
3                                395142003,171231001   
4  173160006,703423002,73761001,418891003,3992080...   

                            Medication Orders  Condition  \
0                               834101,824184  444814009   
1                 313782,198405,310965,757594   38341003   
2  749762,1020137,861467,849574,834060,831533   10509002   
3                 665078,727316,895994,745679  444814009   
4                              1736854,583214   15777000   

                                            Document  
0                            117015009 834101 824184  
1  274474001 66348005 31208007 252160004 313782 1...  
2  699253003 269911007 749762 1020137 861467 8495...  
3    395142003 171231001 665078 727316 895994 745679  
4  173160006 703423002 73761001 418891003 3992080...

In [47]:
# f2 = open("data/disease.txt", 'w')
# for _,doc in condition_df[:train_size].iterrows():
#     f2.write('data/disease/disease-train/'+doc['Condition']+'/'+doc['patient']+'\tdisease-train'+'\t'+doc['Condition']+'\n')
#     with open('data/corpus/disease/disease-train/'+doc['Condition']+'/'+doc['patient'], 'w') as f:
#         f.write(doc['Document'])
# f2.close()

In [48]:
# f2 = open("data/disease.txt", 'a')
# for _,doc in condition_df[-test_size:].iterrows():
#     f2.write('data/disease/disease-test/'+doc['Condition']+'/'+doc['patient']+'\tdisease-test'+'\t'+doc['Condition']+'\n')
#     with open('data/corpus/disease/disease-test/'+doc['Condition']+'/'+doc['patient'], 'w') as f:
#         f.write(doc['Document'])
# f2.close()

# Text-GCN

In [49]:
dataset = 'disease'
word_embeddings_dim = 130

In [50]:
#train_test_split
size = len(resource_df)
train_size = int(0.9*size)
test_size = size - train_size
print(size, train_size, test_size)

143 128 15


In [51]:
#shuffling
train_ids = np.arange(train_size)
test_ids = np.arange(train_size, size)
random.shuffle(train_ids)
random.shuffle(test_ids)
shuffled_id = np.hstack((train_ids,test_ids))
shuffled_document = resource_df['Document'].values[shuffled_id]

In [52]:
# build vocab
word_freq = defaultdict(lambda : 0)
word_set = set()
for doc_words in shuffled_document:
    words = doc_words.split()
    for word in words:
        word_set.add(word)
        word_freq[word] += 1

vocab = list(word_set)
vocab_size = len(vocab)

In [53]:
word_doc_list = defaultdict(lambda : [])
for index, doc in enumerate(shuffled_document):
    appeared = set()
    doc_words = doc.split()
    for word in doc_words:
        if word in appeared:
            continue
        word_doc_list[word].append(index)
        appeared.add(word)
        
word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)

word_id_map = {}
for index, word in enumerate(vocab):
    word_id_map[word] = index

In [54]:
#ecoding using tfidf
tfidf_vec = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vec.fit_transform(vocab)
tfidf_matrix_array = tfidf_matrix.toarray()

### Feature matrix preparation

In [233]:
word_embeddings_dim = 130
word_vector_map = {}
for word, idex in word_id_map.items():
    word_vector_map[word] = tfidf_matrix_array[index]

In [234]:
#train data feature
row_x = []
col_x = []
data_x = []

for i, doc in enumerate(shuffled_document[train_size:]):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = doc.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector) 
            
    for j in range(word_embeddings_dim):
        row_x.append(i)
        col_x.append(j)
        data_x.append(doc_vec[j] / doc_len)
        
x = sp.csr_matrix((data_x, (row_x, col_x)), shape=(
    train_size, word_embeddings_dim))

labels = resource_df['Condition']
labels_list = list(labels.unique())

#train data labels
y = []
for label in labels.values[:train_size]:
    one_hot = [0 for _ in range(len(labels_list))]
    index = labels_list.index(label)
    one_hot[index]= 1
    y.append(one_hot)
    
y = np.array(y)
    

In [235]:
#test data feature
row_tx = []
col_tx = []
data_tx = []

for i, doc in enumerate(shuffled_document[train_size:]):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = doc.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector) 
            
    for j in range(word_embeddings_dim):
        row_tx.append(i)
        col_tx.append(j)
        data_tx.append(doc_vec[j] / doc_len)

# for i in range(vocab_size):
#     for j in range(word_embeddings_dim):
#         row_tx.append(int(i + test_size))
#         col_tx.append(j)
#         data_tx.append(word_vectors.item((i, j))) 
        
tx = sp.csr_matrix((data_tx, (row_tx, col_tx)), shape=(
    test_size, word_embeddings_dim))

#test data label
ty = []
for label in labels.values[train_size:]:
    one_hot = [0 for _ in range(len(labels_list))]
    index = labels_list.index(label)
    one_hot[index]= 1
    ty.append(one_hot)
    
    
# for i in range(vocab_size):
#     one_hot = [0 for _ in range(len(labels_list))]
#     ty.append(one_hot)
ty = np.array(ty)

In [236]:
word_vectors = np.random.uniform(-0.01, 0.01,
                                 (vocab_size, word_embeddings_dim))

for i in range(len(vocab)):
    word = vocab[i]
    if word in word_vector_map:
        vector = word_vector_map[word]
        word_vectors[i] = vector

In [237]:
# allx: the the feature vectors of both labeled and unlabeled training instances
# (a superset of x)
# unlabeled training instances -> words

row_allx = []
col_allx = []
data_allx = []

for i, doc in enumerate(shuffled_document[train_size:]):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = doc.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector) 
            
    for j in range(word_embeddings_dim):
        row_allx.append(i)
        col_allx.append(j)
        data_allx.append(doc_vec[j] / doc_len)

for i in range(vocab_size):
    for j in range(word_embeddings_dim):
        row_allx.append(int(i + train_size))
        col_allx.append(j)
        data_allx.append(word_vectors.item((i, j))) 
        
row_allx = np.array(row_allx)
col_allx = np.array(col_allx)
data_allx = np.array(data_allx)
        
allx = sp.csr_matrix(
    (data_allx, (row_allx, col_allx)), shape=(train_size + vocab_size, word_embeddings_dim))



ally = []
for label in labels.values[:train_size]:
    one_hot = [0 for _ in range(len(labels_list))]
    index = labels_list.index(label)
    one_hot[index]= 1
    ally.append(one_hot)

for i in range(vocab_size):
    one_hot = [0 for l in range(len(labels_list))]
    ally.append(one_hot)

ally = np.array(ally)

print(x.shape, y.shape, tx.shape, ty.shape, allx.shape, ally.shape)

(128, 130) (128, 5) (15, 130) (15, 5) (258, 130) (258, 5)


### Doc word heterogeneous graph

In [238]:
# word co-occurence with context windows
window_size = 2
windows = []

for doc_words in shuffled_document[:train_size]:
    words = doc_words.split()
    length = len(words)
    if length <= window_size:
        windows.append(words)
    else:
        # print(length, length - window_size + 1)
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)

In [239]:
word_window_freq = defaultdict(lambda :0)
for window in windows:
    appeared = set()
    for word in window:
        if word in appeared:
            continue
        word_window_freq[word]+=1
        appeared.add(word)

word_pair_count = defaultdict(lambda :0)
for window in windows:
    for i in range(1, len(window)):
        for j in range(0,i):
            word_i = window[i]
            word_j = window[j]
            word_i_id  = word_id_map[word_i]
            word_j_id =  word_id_map[word_j]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id)
            word_pair_count[word_pair_str]+=1
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            word_pair_count[word_pair_str]+=1

In [244]:
# Word-Word node pmi as weights
row = []
col = []
weight = []
num_window = len(windows)

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[vocab[i]]
    word_freq_j = word_window_freq[vocab[j]]
    pmi = log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    row.append(test_size +i)
    col.append(test_size + j)
    weight.append(pmi)

In [245]:
# doc word frequency
doc_word_freq = defaultdict(lambda : 0)

for doc_id, doc in enumerate(shuffled_document):
    words = doc_words.split()
    for word in words:
        word_id = word_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        doc_word_freq[doc_word_str]+=1

In [246]:
len(row), len(col), node_size, vocab_size, word_embeddings_dim, len(weight)

(794, 794, 145, 130, 130, 794)

In [247]:
#Word-Doc node train weights
for i, doc_words in enumerate(shuffled_document[:train_size]):
    words = doc_words.split()
    doc_word_set = set()
    for word in words:
        if word in doc_word_set:
            continue
        j = word_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        row.append(i)
        col.append(test_size + j)
        idf = log(1.0 * len(shuffled_document) /
                  word_doc_freq[vocab[j]])
        weight.append(freq * idf)
#         print(len(col), len(weight))
        doc_word_set.add(word)

node_size = train_size + vocab_size 

print(len(row),len(col), len(weight), max(row), node_size)
#+ test_size
adj = sp.csr_matrix(
    (weight, (row, col)), shape=(node_size, node_size))



1616 1616 1616 144 258


# Training on GCN

In [251]:
num_node = node_size

In [252]:
edge_list = []
for r,c in zip(row,col):
    edge_list.append((r,c))


feature = allx.toarray().astype('float32')
edge_feature = np.array(weight).reshape(-1,1).astype('float32')

In [253]:
# create a graph
g = pgl.Graph(edges = edge_list,
              num_nodes = num_node,
              node_feat = {'nfeat':feature},
              edge_feat ={'efeat': edge_feature})

In [254]:
print('There are %d nodes in the graph.'%g.num_nodes)
print('There are %d edges in the graph.'%g.num_edges)

There are 258 nodes in the graph.
There are 1616 edges in the graph.


In [255]:
class GCN(nn.Layer):
    """Implement of GCN
    """

    def __init__(self,
                 input_size,
                 num_class,
                 num_layers=2,
                 hidden_size=16,
                 **kwargs):
        super(GCN, self).__init__()
        self.num_class = num_class
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gcns = nn.LayerList()
        for i in range(self.num_layers):
            if i == 0:
                self.gcns.append(
                    pgl.nn.GCNConv(
                        input_size,
                        self.hidden_size,
                        activation="sigmoid",
                        norm=True))
            else:
                self.gcns.append(
                    pgl.nn.GCNConv(
                        self.hidden_size,
                        self.hidden_size,
                        activation="sigmoid",
                        norm=True))

        self.output = nn.Linear(self.hidden_size, self.num_class)
    def forward(self, graph, feature):
        for m in self.gcns:
            feature = m(graph, feature)
        logits = self.output(feature)
        return logits

In [256]:
label = np.array(ally, dtype="float32")
g = g.tensor()
y = paddle.to_tensor(label)
gcn = GCN(130, 5)
criterion = paddle.nn.loss.CrossEntropyLoss(soft_label=True)
optim = Adam(learning_rate=0.01,
             parameters=gcn.parameters())

In [257]:
gcn.train()
for epoch in range(3000):
    logits = gcn(g, g.node_feat['nfeat'])
#     print(logits[:10])
    loss = criterion(logits, y)
    loss.backward()
    optim.step()
    optim.clear_grad()
    if epoch%100 == 0:
        print("epoch: %s | loss: %.4f" % (epoch, loss.numpy()[0]))

epoch: 0 | loss: 0.8250
epoch: 100 | loss: 0.7460
epoch: 200 | loss: 0.7475
epoch: 300 | loss: 0.7490
epoch: 400 | loss: 0.7494
epoch: 500 | loss: 0.7496
epoch: 600 | loss: 0.7497
epoch: 700 | loss: 0.7497
epoch: 800 | loss: 0.7497
epoch: 900 | loss: 0.7497
epoch: 1000 | loss: 0.7497
epoch: 1100 | loss: 0.7497
epoch: 1200 | loss: 0.7497
epoch: 1300 | loss: 0.7497
epoch: 1400 | loss: 0.7497
epoch: 1500 | loss: 0.7497
epoch: 1600 | loss: 0.7497
epoch: 1700 | loss: 0.7497
epoch: 1800 | loss: 0.7497
epoch: 1900 | loss: 0.7497
epoch: 2000 | loss: 0.7497
epoch: 2100 | loss: 0.7497
epoch: 2200 | loss: 0.7497
epoch: 2300 | loss: 0.7497
epoch: 2400 | loss: 0.7498
epoch: 2500 | loss: 0.7497
epoch: 2600 | loss: 0.7497
epoch: 2700 | loss: 0.7497
epoch: 2800 | loss: 0.7497
epoch: 2900 | loss: 0.7497


### Test accuracy

In [228]:
x_test = paddle.to_tensor(tx.toarray().astype('float32'))

In [ ]:
g_test = pgl.Graph(edges = edge_list,
              num_nodes = num_node,
              node_feat = {'nfeat':x_test},
              edge_feat ={'efeat': edge_feature})

In [229]:
logits = gcn.forward(g.tensor(), x_test)